In [1]:
import getpass

import fsspec
import pandas as pd

In [2]:
import barqs
import fasta
import fastq

In [3]:
_password = getpass.getpass(prompt="Enter password for ssh host: ")

Enter password for ssh host:  ········


In [4]:
fs = fsspec.filesystem("ssh", host="gesu.nygenome.org", username="collinsd", password=_password)
remote_file = fs.open("/brahms/kowalskim/data/for_john/combined.R1.fastq.gz")

In [5]:
raw_reads = fastq.load(remote_file)

In [6]:
first_read = next(raw_reads)

KeyboardInterrupt: 

In [10]:
umi_length = 9
barcode_length = 12

In [22]:
reads = (
    barqs.extract(
        read, 
        umi_length=umi_length, 
        barcode_length=barcode_length
    )
    for read in raw_reads
)

In [23]:
features = [
    ("DAVE", "GATGCGGTGGAA"),
    ("ELIA", "GAACTGATTGCG"),
]

In [24]:
trimmed_reads = (
    barqs.trim(read, features=features)
    for read in reads
)

In [25]:
observed_features = barqs.filter_duplicates(trimmed_reads)

In [26]:
counts = barqs.quantify(observed_features, features)

In [32]:
counts_df = pd.DataFrame(counts)

In [33]:
counts_df

,ACGTACGTAGCT,TGCACTGACTGA
DAVE,1,NaN
ELIA,1,2.0


In [11]:
from Bio.Seq import translate

In [12]:
barcodes = ["ACGTACGTAGCT", "TGCACTGACTGA"]
umis = ["ACGTAGCTA", "TGCATGACT"]

In [13]:
subjects = dict(
    dave="GATGCGGTGGAA",
    elia="GAACTGATTGCG",
    no_one="TATACGCGAGGA",
)

verbs = dict(likes="CTGATTAAAGAAAGC")

objects = dict(reading="CGCGAAGCGGATATTAACGGC")


In [14]:
seq1 = barcode1 + umi1 + dave + likes + reading

seq1

'ACGTACGTAGCTACGTAGCTAGATGCGGTGGAACTGATTAAAGAAAGCCGCGAAGCGGATATTAACGGC'

In [15]:
seq2 = barcode1 + umi1 + dave + likes + reading

seq2

'ACGTACGTAGCTACGTAGCTAGATGCGGTGGAACTGATTAAAGAAAGCCGCGAAGCGGATATTAACGGC'

In [16]:
seq3 = barcode1 + umi2 + elia + likes + reading

seq3

'ACGTACGTAGCTTGCATGACTGAACTGATTGCGCTGATTAAAGAAAGCCGCGAAGCGGATATTAACGGC'

In [17]:
seq4 = barcode2 + umi1 + elia + likes + reading

seq4

'TGCACTGACTGAACGTAGCTAGAACTGATTGCGCTGATTAAAGAAAGCCGCGAAGCGGATATTAACGGC'

In [18]:
seq5 = barcode2 + umi2 + elia + likes + reading

seq5

'TGCACTGACTGATGCATGACTGAACTGATTGCGCTGATTAAAGAAAGCCGCGAAGCGGATATTAACGGC'

In [19]:
seq6 = barcode2 + umi2 + no_one + likes + reading

seq6

'TGCACTGACTGATGCATGACTTATACGCGAGGACTGATTAAAGAAAGCCGCGAAGCGGATATTAACGGC'

In [39]:
len("GATGCGGTGGAA")

12